In [37]:

import json
import sys
import os
import time
import numpy as np
import cv2
import onnx
import onnxruntime
from matplotlib import pyplot as plt
from onnx import numpy_helper



In [38]:
model_dir = "./mnist"
model = model_dir + "/model.onnx"


In [43]:
def read_image(img):
    #img = np.dot(img[..., :3], [0.299, 0.587, 0.114])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    img = cv2.resize(img, dsize=(28, 28), interpolation=cv2.INTER_AREA)
    #img = cv2.transform(img, np.array([[0.299], [0.587], [0.114]]))

    img.resize((1, 1, 28, 28))

    data = json.dumps({'data': img.tolist()})
    data = np.array(json.loads(data)['data']).astype('float32')

    session = onnxruntime.InferenceSession(model, None)
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name

    result = session.run([output_name], {input_name: data})
    prediction=int(np.argmax(np.array(result).squeeze(), axis=0))
    return prediction

In [44]:
from PIL import Image
from IPython.display import display
for index, file_name in enumerate(os.listdir('mnist-test')):
    if index > 10:
        break
    full_filename = os.path.join('mnist-test', file_name)
    img = cv2.imread(full_filename)
  
    #plt.imshow(img)
    result = read_image(img)
    print(result)
    img = img[:,:,::-1]
    display(Image.fromarray(img))
    

2


1


0


1


1


...

1


1


0


7


5


In [45]:
session = onnxruntime.InferenceSession(model)

# Get inputs and outputs
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

print('Input Name:', input_name)
print('Output Name:', output_name)

# Print input shape and type
print('Input Shape:', session.get_inputs()[0].shape)
print('Input Type:', session.get_inputs()[0].type)

# Print output shape and type
print('Output Shape:', session.get_outputs()[0].shape)
print('Output Type:', session.get_outputs()[0].type)

Input Name: Input3
Output Name: Plus214_Output_0
Input Shape: [1, 1, 28, 28]
Input Type: tensor(float)
Output Shape: [1, 10]
Output Type: tensor(float)


In [52]:
def read_images_batch(imgs):
    processed_imgs = []
    for img in imgs:
        img = np.dot(img[..., :3], [0.299, 0.587, 0.114])
        img = cv2.resize(img, dsize=(28, 28), interpolation=cv2.INTER_AREA)
        img = np.resize(img, (1, 28, 28))
        processed_imgs.append(img)

    data = np.array(processed_imgs).astype('float32')
    data = np.reshape(data, (-1, 1, 28, 28))

    session = onnxruntime.InferenceSession(model, None)
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name

    result = session.run([output_name], {input_name: data})
    prediction = np.argmax(np.array(result).squeeze(), axis=0)
    return prediction


In [53]:
all_imgs = []
for index, file_name in enumerate(os.listdir('mnist-test')):
    if index > 10:
        break
        
    full_filename = os.path.join('mnist-test', file_name)
    img = cv2.imread(full_filename)
    all_imgs.append(img)
read_images_batch(all_imgs)

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Got invalid dimensions for input: Input3 for the following indices
 index: 0 Got: 11 Expected: 1
 Please fix either the inputs or the model.